# HW07 — Кластеризация и внутренние метрики

## Установка

Для запуска ноутбука нужны:

- `pandas`, `numpy`, `matplotlib`
- `scikit-learn`

Если используете `uv`, то зависимости ставятся так:

```bash
cd <корень-репозитория>
uv sync
```


In [ ]:
from pathlib import Path
from IPython.display import display
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
BASE_DIR = Path('homeworks/HW07') if Path('homeworks/HW07').exists() else Path('.')
DATA_DIR = BASE_DIR / 'data'
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
FIGURES_DIR = ARTIFACTS_DIR / 'figures'
LABELS_DIR = ARTIFACTS_DIR / 'labels'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
LABELS_DIR.mkdir(parents=True, exist_ok=True)

## 1. Датасеты и параметры

Выбраны датасеты 01, 02 и 04.


In [ ]:
DATASETS = {
    'ds1': {
        'file': 'S07-hw-dataset-01.csv',
        'k_range': list(range(2, 11)),
        'dbscan_eps': [0.3, 0.5, 0.7, 1.0, 1.5],
        'dbscan_min_samples': [5, 10],
    },
    'ds2': {
        'file': 'S07-hw-dataset-02.csv',
        'k_range': list(range(2, 11)),
        'dbscan_eps': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0],
        'dbscan_min_samples': [5, 10, 20],
    },
    'ds4': {
        'file': 'S07-hw-dataset-04.csv',
        'k_range': list(range(2, 11)),
        'dbscan_eps': [0.5, 0.8, 1.0, 1.2, 1.5, 2.0, 2.5],
        'dbscan_min_samples': [5, 10, 20],
    },
}

## 2. Вспомогательные функции


In [ ]:
def make_one_hot_encoder():
    try:
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown='ignore', sparse=False)
def build_preprocessor(df):
    feature_cols = [c for c in df.columns if c != 'sample_id']
    X = df[feature_cols]
    cat_cols = [c for c in X.columns if X[c].dtype == 'object']
    num_cols = [c for c in X.columns if c not in cat_cols]

    numeric_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ])
    if cat_cols:
        categorical_pipe = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', make_one_hot_encoder()),
        ])
        preprocessor = ColumnTransformer([
            ('num', numeric_pipe, num_cols),
            ('cat', categorical_pipe, cat_cols),
        ])
    else:
        preprocessor = ColumnTransformer([
            ('num', numeric_pipe, num_cols),
        ])
    return preprocessor, X
    
def compute_metrics(X, labels):
    if len(np.unique(labels)) < 2:
        return None
    return {
        'silhouette': float(silhouette_score(X, labels)),
        'davies_bouldin': float(davies_bouldin_score(X, labels)),
        'calinski_harabasz': float(calinski_harabasz_score(X, labels)),
    }
def compute_dbscan_metrics(X, labels):
    noise_mask = labels == -1
    noise_share = float(noise_mask.mean())
    labels_non_noise = labels[~noise_mask]
    X_non_noise = X[~noise_mask]
    if len(np.unique(labels_non_noise)) < 2:
        return None, noise_share
    metrics = {
        'silhouette': float(silhouette_score(X_non_noise, labels_non_noise)),
        'davies_bouldin': float(davies_bouldin_score(X_non_noise, labels_non_noise)),
        'calinski_harabasz': float(calinski_harabasz_score(X_non_noise, labels_non_noise)),
    }
    return metrics, noise_share

## 3. Загрузка и первичный анализ (для каждого датасета)


In [ ]:
for ds_key, cfg in DATASETS.items():
    df = pd.read_csv(DATA_DIR / cfg['file'])
    print('=' * 80)
    print(ds_key, cfg['file'])
    display(df.head())
    display(df.info())
    display(df.describe(include='all'))
    missing = df.isna().sum().sort_values(ascending=False)
    missing_share = (missing / len(df)).round(3)
    display(pd.DataFrame({'missing': missing, 'share': missing_share}).head(10))

## 4. Обучение моделей, метрики и визуализация

Для каждого датасета: KMeans (подбор k), DBSCAN (подбор eps и min_samples)
Метрики: silhouette / Davies-Bouldin / Calinski-Harabasz,
PCA(2D) для лучшего решения


In [ ]:
metrics_summary = {}
best_configs = {}

# тут основной цикл по датасетам
for ds_key, cfg in DATASETS.items():
    df = pd.read_csv(DATA_DIR / cfg['file'])
    preprocessor, X_raw = build_preprocessor(df)
    X = preprocessor.fit_transform(X_raw)

    print('\n' + '=' * 80)
    print(f'{ds_key}: {cfg["file"]}')

    ds_metrics = {'kmeans': {}, 'dbscan': {}}

    # KMeans: перебираем k
    k_list = []
    for k in cfg['k_range']:
        model = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = model.fit_predict(X)
        metrics = compute_metrics(X, labels)
        ds_metrics['kmeans'][str(k)] = metrics
        k_list.append((k, metrics['silhouette'] if metrics else -1))

    ks = [k for k, _ in k_list]
    sils = [v for _, v in k_list]
    plt.figure(figsize=(6, 4))
    plt.plot(ks, sils, marker='o')
    plt.title(f'{ds_key.upper()}: KMeans silhouette vs k')
    plt.xlabel('k')
    plt.ylabel('silhouette')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / f'{ds_key}_kmeans_silhouette_vs_k.png', dpi=150)
    plt.close()

    best_k = max(k_list, key=lambda x: x[1])[0]

    # DBSCAN: eps/min_samples
    best_dbscan = None
    for eps in cfg['dbscan_eps']:
        for min_samples in cfg['dbscan_min_samples']:
            model = DBSCAN(eps=eps, min_samples=min_samples)
            labels = model.fit_predict(X)
            metrics, noise_share = compute_dbscan_metrics(X, labels)
            key = f'eps={eps},min_samples={min_samples}'
            ds_metrics['dbscan'][key] = {'metrics': metrics, 'noise_share': noise_share}
            if metrics is None or noise_share > 0.3:
                continue
            score = metrics['silhouette']
            if best_dbscan is None or score > best_dbscan['score']:
                best_dbscan = {
                    'eps': eps,
                    'min_samples': min_samples,
                    'score': score,
                    'metrics': metrics,
                    'noise_share': noise_share,
                }

    best_k_metrics = ds_metrics['kmeans'][str(best_k)]
    best_method = 'kmeans'
    best_params = {'k': int(best_k)}
    best_metrics = best_k_metrics
    best_noise = None

    if best_dbscan and best_dbscan['score'] > (best_k_metrics['silhouette'] if best_k_metrics else -1):
        best_method = 'dbscan'
        best_params = {'eps': best_dbscan['eps'], 'min_samples': best_dbscan['min_samples']}
        best_metrics = best_dbscan['metrics']
        best_noise = best_dbscan['noise_share']

    # финальная модель, чтобы сохранить метки
    if best_method == 'kmeans':
        best_model = KMeans(n_clusters=best_params['k'], random_state=42, n_init=10)
        labels = best_model.fit_predict(X)
    else:
        best_model = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
        labels = best_model.fit_predict(X)

    labels_path = LABELS_DIR / f'labels_hw07_{ds_key}.csv'
    pd.DataFrame({'sample_id': df['sample_id'], 'cluster_label': labels}).to_csv(labels_path, index=False)

    # PCA для картинки
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X)
    plt.figure(figsize=(6, 4))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, s=8, cmap='tab10', alpha=0.75)
    plt.title(f'{ds_key.upper()}: PCA(2D) best ({best_method})')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / f'{ds_key}_pca_best.png', dpi=150)
    plt.close()

    metrics_summary[ds_key] = {
        'dataset_file': cfg['file'],
        'kmeans': ds_metrics['kmeans'],
        'dbscan': ds_metrics['dbscan'],
        'best_method': best_method,
        'best_metrics': best_metrics,
        'best_noise_share': best_noise,
    }

    best_configs[ds_key] = {
        'dataset_file': cfg['file'],
        'best_method': best_method,
        'best_params': best_params,
        'selection_criterion': 'max silhouette (DBSCAN only if noise_share <= 0.3)',
    }

    print('Best method:', best_method)
    print('Best params:', best_params)
    print('Best metrics:', best_metrics)
    if best_method == 'dbscan':
        print('Noise share:', best_noise)


## 5. Устойчивость (KMeans, датасет ds1)


In [ ]:
cfg = DATASETS['ds1']
df = pd.read_csv(DATA_DIR / cfg['file'])
preprocessor, X_raw = build_preprocessor(df)
X = preprocessor.fit_transform(X_raw)

best_k_ds1 = best_configs['ds1']['best_params'].get('k', 2)

seeds = [0, 1, 2, 3, 4]
labels_by_seed = []
for seed in seeds:
    model = KMeans(n_clusters=int(best_k_ds1), random_state=seed, n_init=10)
    labels_by_seed.append(model.fit_predict(X))

pairwise_ari = []
for i in range(len(seeds)):
    for j in range(i + 1, len(seeds)):
        pairwise_ari.append(adjusted_rand_score(labels_by_seed[i], labels_by_seed[j]))

stability = {
    'dataset': 'ds1',
    'kmeans_k': int(best_k_ds1),
    'seeds': seeds,
    'pairwise_ari_mean': float(np.mean(pairwise_ari)),
    'pairwise_ari_min': float(np.min(pairwise_ari)),
    'pairwise_ari_max': float(np.max(pairwise_ari)),
}

stability


## 6. Сохранение артефактов


In [ ]:
(ARTIFACTS_DIR / 'metrics_summary.json').write_text(json.dumps(metrics_summary, indent=2))
(ARTIFACTS_DIR / 'best_configs.json').write_text(json.dumps(best_configs, indent=2))
(ARTIFACTS_DIR / 'stability_kmeans_ds1.json').write_text(json.dumps(stability, indent=2))

## 7. Итоговые выводы по датасетам

**ds1 (dataset-01)**
- После масштабирования KMeans с небольшим числом кластеров дал лучший silhouette. DBSCAN работал стабильнее при большем eps, уступает по метрикам. Явные различия масштабов критичны: без scaling результат заметно хуже.

**ds2 (dataset-02)**
- Нелинейная структура лучше выделилась через DBSCAN при eps=1.0 и min_samples=20. При корректном eps шум минимален, а silhouette заметно выше, чем у KMeans. KMeans чувствителен к форме кластеров, дает срез для нелинейных групп.

**ds4 (dataset-04)**
- Использовался имьютинг числовых и one-hot для категориальных признаков. DBSCAN с eps=2.5 и min_samples=20 дал допустимыый баланс метрик и шума. Высокая размерность снижает контраст кластеров, метрики умеренные.
